In [9]:
!pip install paho-mqtt


In [10]:
import random
import time
import paho.mqtt.client as mqtt

# Station identity
STATION_ID = "station001"

# MQTT Broker (public HiveMQ broker for testing)
BROKER = "broker.hivemq.com"
PORT = 1883
TOPIC_BASE = f"iot/{STATION_ID}"

# Create MQTT client and connect
client = mqtt.Client()
client.connect(BROKER, PORT)


C:\Users\Sanket\AppData\Local\Temp\ipykernel_18960\952978718.py:14: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [11]:
def generate_sensor_data():
    temperature = round(random.uniform(-50, 50), 2)
    humidity = round(random.uniform(0, 100), 2)
    co2 = round(random.uniform(300, 2000), 2)
    return temperature, humidity, co2


In [12]:
# Publish a single set of data (run this cell multiple times to test)
temp, hum, co2 = generate_sensor_data()
payload = f"{temp},{hum},{co2}"
topic = f"{TOPIC_BASE}/environment"
print(f"Publishing to {topic}: {payload}")
client.publish(topic, payload)


Publishing to iot/station001/environment: 1.88,5.52,1742.43


In [13]:
import requests

THINGSPEAK_API_KEY = "6XWQWLVQ7Y74R6G6"

def post_to_thingspeak(temp, hum, co2):
    url = "https://api.thingspeak.com/update"
    payload = {
        "api_key": THINGSPEAK_API_KEY,
        "field1": temp,
        "field2": hum,
        "field3": co2
    }
    response = requests.post(url, data=payload)
    print("Uploaded to ThingSpeak:", response.status_code)


In [ ]:
# Publish sensor data every 5 seconds (use Ctrl + I to interrupt in Jupyter)
try:
    while True:
        temp, hum, co2 = generate_sensor_data()
        payload = f"{temp},{hum},{co2}"
        topic = f"{TOPIC_BASE}/environment"
        print(f"Publishing to {topic}: {payload}")
        client.publish(topic, payload)
        post_to_thingspeak(temp, hum, co2)
        time.sleep(5)
except KeyboardInterrupt:
    print("Stopped publishing.")


Publishing to iot/station001/environment: -22.35,45.21,1414.43
Uploaded to ThingSpeak: 200
Publishing to iot/station001/environment: -37.21,72.58,548.49
Uploaded to ThingSpeak: 200
Publishing to iot/station001/environment: 29.09,86.02,1215.13
Uploaded to ThingSpeak: 200
Publishing to iot/station001/environment: 31.0,47.65,1936.76
Uploaded to ThingSpeak: 200
Publishing to iot/station001/environment: 30.02,78.44,1155.87
Uploaded to ThingSpeak: 200


In [ ]:
import paho.mqtt.client as mqtt

# Define callback for when a message is received
def on_message(client, userdata, msg):
    print(f"Received message from topic `{msg.topic}`: {msg.payload.decode()}")

# Create MQTT client and configure callbacks
subscriber = mqtt.Client()
subscriber.on_message = on_message

# Connect to the same broker
subscriber.connect("broker.hivemq.com", 1883)

# Subscribe to the topic
subscriber.subscribe("iot/station001/environment")

# Start listening for messages (non-blocking)
subscriber.loop_start()
